# Assignments

## Assignment - 1

In [112]:
import findspark

findspark.init()

In [87]:
import pyspark

from pyspark.sql import SparkSession

from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
spark = SparkSession.builder.master("local").appName("SparkApp").getOrCreate()

# Find min and max temperatures in given weather_data
### *The data is seperated by one or more spaces. So first transform the data and do the calculations*

In [4]:
path  = "D:/futurense_hadoop-pyspark/labs/dataset/weather/weather_data.txt"

In [30]:
columns = ['WBANNO',
 'LST_DATE',
 'CRX_VN',
 'LONGITUDE',
 'LATITUDE',
 'T_DAILY_MAX',
 'T_DAILY_MIN',
 'T_DAILY_MEAN',
 'T_DAILY_AVG',
 'P_DAILY_CALC',
 'SOLARAD_DAILY',
 'SUR_TEMP_DAILY_TYPE',
 'SUR_TEMP_DAILY_MAX',
 'SUR_TEMP_DAILY_MIN',
 'SUR_TEMP_DAILY_AVG',
 'RH_DAILY_MAX',
 'RH_DAILY_MIN',
 'RH_DAILY_AVG',
 'SOIL_MOISTURE_5_DAILY',
 'SOIL_MOISTURE_10_DAILY',
 'SOIL_MOISTURE_20_DAILY',
 'SOIL_MOISTURE_50_DAILY',
 'SOIL_MOISTURE_100_DAILY',
 'SOIL_TEMP_5_DAILY',
 'SOIL_TEMP_10_DAILY',
 'SOIL_TEMP_20_DAILY',
 'SOIL_TEMP_50_DAILY',
 'SOIL_TEMP_100_DAILY']


### First read the data remove extra spaces and make it as comma seperated values

In [73]:
data = spark.read.load(path,format="csv")

#df = data.withColumn("values", regexp_replace(data.values, "\s+", ",")).withColumn("values",split(data.value,","))

df = data.withColumn("_c0",regexp_replace(data._c0,"\s+",","))

### Split the data based on ',' and store values in respective columns

In [75]:

split_data = df.select(split(df._c0, ",").alias("data"))


converted_data = split_data.select(
    *[split_data.data.getItem(idx).alias(columns[idx]) for idx in range(len(columns))])
    

#select the required the columns and type cast them into required datatype     
min_max_df = converted_data.select(col("T_DAILY_MAX").cast(FloatType()).alias("T_DAILY_MAX"),col("T_DAILY_MIN").cast(FloatType()).alias("T_DAILY_MIN"))

# find the min from the daily_min and max from daily_max
min_max_df.agg(min("T_DAILY_MIN").alias("Min_temp"),max("T_DAILY_MAX").alias("max_temp")).show()

+--------+--------+
|Min_temp|max_temp|
+--------+--------+
|    -7.9|    36.0|
+--------+--------+



### Find the max and min temperatures in each month

In [116]:
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

def find_month_name(month_num):
    return months[int(month_num) - 1]

month_df = udf(find_month_name,StringType())


df = converted_data.withColumn('MONTH',month_df(substring('LST_DATE',5,2)))\
                   .select(
                            col('MONTH'),col("T_DAILY_MAX").cast(FloatType()).alias("T_DAILY_MAX"),
                            col("T_DAILY_MIN").cast(FloatType()).alias("T_DAILY_MIN")
                          )

df.groupBy('MONTH').agg(min('T_DAILY_MIN').alias('Min_temp'),max('T_DAILY_MAX').alias('Max_temp')).show()

+-----+--------+--------+
|MONTH|Min_temp|Max_temp|
+-----+--------+--------+
|  May|    14.3|    31.1|
|  Jun|     0.0|    33.6|
|  Feb|    -3.5|    26.6|
|  Mar|    -3.2|    29.1|
|  Jan|    -7.9|    26.5|
|  Apr|     8.0|    30.8|
|  Jul|    19.8|    36.0|
+-----+--------+--------+

